In [36]:
# importing libraries
import torch
import tiktoken
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [11]:
# read the text file
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

In [23]:
# feeding the text and cleaning it
file_path = 'training_text.txt'
text = read_text_file(file_path)

text = text.strip("\ufeff")
text = text.replace("\n", " ")

In [29]:
# length of text
print(len(text))
print(text[:100])

In my younger and more vulnerable years my father gave me some advice that I’ve been turning over in


In [19]:
# checking all the characters used in the text
char = sorted(list(set(text)))
vocab_size = len(char)
print(''.join(char))
print(vocab_size)


 !$()*,-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXY[]abcdefghijklmnopqrstuvwxyzçéêô —‘’“”…
87


In [20]:
# bringing in tiktoken's tokenizer
enc = tiktoken.get_encoding("gpt2")
enc.n_vocab

50257

In [25]:
# testing with encoding and decoding; it works
encode = enc.encode("hello")
print(encode)
enc.decode(encode)

encode1 = enc.encode(text[:100])
print(encode1)
enc.decode(encode1)

[31373]
[818, 616, 7099, 290, 517, 8826, 812, 616, 2988, 2921, 502, 617, 5608, 326, 314, 447, 247, 303, 587, 6225, 625, 287]


'In my younger and more vulnerable years my father gave me some advice that I’ve been turning over in'

In [33]:
# now encoding the entire text
data = torch.tensor(enc.encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:500])

torch.Size([71784]) torch.int64
tensor([  818,   616,  7099,   290,   517,  8826,   812,   616,  2988,  2921,
          502,   617,  5608,   326,   314,   447,   247,   303,   587,  6225,
          625,   287,   616,  2000,  1683,  1201,    13,   220,   564,   250,
        28877,   345,  1254,   588, 26816,  2687,    11,   447,   251,   339,
         1297,   502,    11,   564,   250,  3137,  3505,   326,   477,   262,
          661,   287,   428,   995,  4398,   447,   247,    83,   550,   262,
        13391,   326,   345,   447,   247,   303,   550,    13,   447,   251,
          220,   679,  1422,   447,   247,    83,   910,   597,   517,    11,
          475,   356,   447,   247,   303,  1464,   587, 23708,  1316, 43058,
          287,   257, 10395,   835,    11,   290,   314,  7247,   326,   339,
         4001,   257,  1049,  1730,   517,   621,   326,    13,   554, 12921,
           11,   314,   447,   247,    76, 19514,   284, 11515,   477,  2553,
        43547,    11,   257,  79

In [34]:
# splitting data into training data and validation data
n = int(0.9*len(data))
train = data[:n]
val = data[n:]

In [35]:
# setting up block size
block_size = 8
train[:block_size + 1]

tensor([ 818,  616, 7099,  290,  517, 8826,  812,  616, 2988])

In [45]:
# setting batch size
batch_size = 4

# function for getting a batch of random blocks within data, set my batch_size
def get_batch(split):
    if split == "train":
        data = train
    else:
        data = val

    ix = torch.randint(len(data) - batch_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    
    return x, y

In [50]:
# collecting inputs and targets from the training data
# targets used for creating the loss function later on
xb, yb = get_batch("train")

print(xb)
print(yb)

tensor([[ 7953,   284,   257,   582,   422,   968, 12255,    13],
        [ 2300,    30,   447,   251,   220,   564,   250,    40],
        [  487,  1108,    13,   220,  8127,   447,   247,    82],
        [   13,   564,   250,  6090,   447,   247,    83,   345]])
tensor([[  284,   257,   582,   422,   968, 12255,    13,   554],
        [   30,   447,   251,   220,   564,   250,    40,   447],
        [ 1108,    13,   220,  8127,   447,   247,    82,  2951],
        [  564,   250,  6090,   447,   247,    83,   345,  1561]])
